In [1]:
import pandas as pd
import numpy as np
import folium
import requests

from bs4 import BeautifulSoup

# pip install tqdm : progressBar 구현
from tqdm.notebook import tqdm 
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# jupyter nbconvert --to script coffeeStore.ipynb

In [2]:
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [3]:
driver.maximize_window() # 윈도우 창 최대화

In [5]:
# 스타벅스) 매장 찾기-지역 검색 링크 주소
starbucks_url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(starbucks_url) # 해당 페이지로 이동하기

In [6]:
# 스타벅스) '서울' 링크 클릭
starbucks_seoul_selector = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_selector))).click()

In [7]:
# 스타벅스) '서울'-'전체' 클릭
starbuck_seoul_all = '#mCSB_2_container > ul > li:nth-child(1) > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbuck_seoul_all))).click()

In [8]:
# 스타벅스 html 코드를 파싱하여 html 파일에 기록합니다.
html = driver.page_source # 해당 페이지의 소스 코드 반환
filename = 'starbucks.html'
htmlfile = open(filename, 'w', encoding='UTF-8')
print(html, file=htmlfile)
htmlfile.close()
print(filename + ' 파일 생성됨')

starbucks.html 파일 생성됨


In [9]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# 파싱된 결과를 Beautiful Soup 객체로 생성합니다.
soup = BeautifulSoup(html, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [10]:
# 매장 정보들을 담고 있는 컨테이너 박스를 찾습니다.
# 컨테이너는 div 태그이고, id가 mCSB_3_container입니다.
container = soup.find('div', id='mCSB_3_container')
storeAll = container.find_all('li')
print('storeAll : %d' % len(storeAll))

storeAll : 615


In [28]:
starbucksData = []

for store in storeAll:
    # print(store)
    brand = '스타벅스'
    name = store.find('strong').text.strip()
    address = store.find('p').text.strip().replace('1522-3232', '')
    imsi = address.split(' ')
    sido = imsi[0]
    gu = imsi[1]
    latitude = store['data-lat'] # 위도
    longitude = store['data-long'] # 경도

    starbucksData.append([brand, name, address, sido, gu, latitude, longitude])
    # break

print(len(starbucksData))

615


In [30]:
sbDataFrame = pd.DataFrame(starbucksData)
sbDataFrame.columns = ['브랜드', '상호', '주소', '시도', '군구', '위도', '경도']
sbDataFrame.head(n=7) # 반대 메소드 tail

,브랜드,상호,주소,시도,군구,위도,경도
0,스타벅스,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),서울특별시,강남구,37.501087,127.043069
1,스타벅스,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),서울특별시,강남구,37.510178,127.022223
2,스타벅스,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),서울특별시,강남구,37.5139309,127.0206057
3,스타벅스,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),서울특별시,강남구,37.499517,127.031495
4,스타벅스,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),서울특별시,강남구,37.494668,127.062583
5,스타벅스,봉은사역,서울특별시 강남구 봉은사로 619 (삼성동),서울특별시,강남구,37.515000,127.063196
6,스타벅스,압구정윤성빌딩,서울특별시 강남구 논현로 834 (신사동),서울특별시,강남구,37.5227934,127.0286009


In [13]:
print('위도 누락 데이터 개수 : %d' % sbDataFrame['위도'].isnull().sum())
print('경도 누락 데이터 개수 : %d' % sbDataFrame['경도'].isnull().sum())

위도 누락 데이터 개수 : 0
경도 누락 데이터 개수 : 0


In [14]:
guList = list(sbDataFrame['군구'].unique())
print('서울시 구 목록 개수 : %d' % len(guList))
print(guList)

서울시 구 목록 개수 : 25
['강남구', '강북구', '강서구', '관악구', '광진구', '금천구', '노원구', '도봉구', '동작구', '마포구', '서대문구', '서초구', '성북구', '송파구', '양천구', '영등포구', '은평구', '종로구', '중구', '강동구', '구로구', '동대문구', '성동구', '용산구', '중랑구']


In [15]:
# 이디야 매장
ediya_url = 'https://ediya.com/contents/find_store.html'
driver.get(ediya_url)

In [16]:
# 이디야 '매장명'과 '주소' 검색에서 '주소' 클릭
ediya_address_selector = '#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_address_selector))).click()

In [17]:
# 누락된 위도와 경도 정보는 kakao api를 이용하여 채워 넣도록 합니다.
# https://developers.kakao.com/
# kakao에 로그인하여 API 키 발급 받기

In [18]:
url_header = 'https://dapi.kakao.com/v2/local/search/address.json?query='
api_key = '16f486bc90ddf8c917b0e903b6bb3ceb'
header = {'Authorization': 'KakaoAK ' + api_key}

In [19]:
# 주소를 입력 받아서 위도와 경도를 반환해주는 함수 구현하기
def getGeoCoder(address):
    result = ""
    url = url_header + address
    response = requests.get(url, headers=header)
    # print(response) # 성공시 <Response [200]>으로 리턴 됨
    # print(response.json())
    if response.status_code == 200:
        try:
            result_address = response.json()["documents"][0]["address"]
            result = result_address["y"], result_address["x"]
        except Exception as err:
            return None
    else:
        result = "ERROR[" + str(response.status_code) + "]"

    return result
# end def   

In [20]:
# 한 개의 매장에 대한 테스트를 진행합니다.
# geoInfo = getGeoCoder('서울 중랑구 망우로 460 (망우동)') # 잘 동작하는 주소
geoInfo = getGeoCoder('서울 노원구 한글비석로 409 (상계동) 1~2층') # NoneType이 리턴되는 주소
geoInfo

In [31]:
ediyaData = []

for gu in tqdm(guList):
    ediya_search_keyword_css = '#keyword'
    
    # 이디야 주소 검색어 초기화
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_keyword_css))).clear()
    
    # 이디야 주소 검색어 입력
    # f"서울 {gu}" : f-string 기법, 서울 중구, 서울 마포구 등등
    WebDriverWait(driver, 10).until(EC.presence_of_element_located
                                    ((By.CSS_SELECTOR, ediya_search_keyword_css))).send_keys(f"서울 {gu}")

    # 이디야 주소 검색 버튼 클릭
    ediya_search_button_css = '#keyword_div > form > button'
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_button_css))).click()

    html = driver.page_source # 각 구별 매장 정보를 담고 있는 html 소스

    # 개별 구에 대한 결과를 파일로 저장해 봅니다.
    # filename = open(f'서울 {gu}.html', 'w', encoding='UTF-8')
    # print(html, file=filename)
    # filename.close()
    # print(f'서울 {gu}.html 파일 생성' )

    soup = BeautifulSoup(html, 'html.parser')
    ul_tag = soup.find('ul', id='placesList')
    
    oneGuEdiyaList = ul_tag.find_all('li')
    for store in oneGuEdiyaList:
        # print(store)
        brand = '이디야'    
        name = store.find('dt').text.strip()
        address = store.find('dd').text.strip()
        imsi = address.split(' ')
        sido = imsi[0]
        gu = imsi[1]

        # 위도/경도 정보가 들어 있는 문자열
        geoInfoString = store.find('a')['onclick']
        geoInfoImsi = geoInfoString # 중간에 변형이 이루어 지므로 임시 복사

        # 넘파이의 nan으로 무의미한 데이터 만들기
        latitude = np.nan # 위도 
        longitude = np.nan # 경도 
        latLong = ['0', '0']

        if geoInfoString.startswith("panLatTo"): 
            # 올바른 위도/경도가 아님
            try:
                if geoInfoString.index(r"panLatTo('0','0'") == 0: 
                    geoInfo = getGeoCoder(address) 
                    if geoInfo != None:
                        latitude = geoInfo[0] # 위도
                        longitude = geoInfo[1] # 경도
                    else:
                        print(address)                
            except ValueError as err:
                # 올바른 위도/경도입니다.
                latLong = geoInfoString.replace(r"panLatTo('", '').replace(r"');fnMove();", '')
                latLong = latLong.split("','")
        
                latitude = latLong[1] 
                longitude = latLong[0]                
            # end try

        else: # 'panAddTo'으로 시작하는 경우
            geoInfo = getGeoCoder(address) 
            if geoInfo != None:
                latitude = geoInfo[0] # 위도
                longitude = geoInfo[1] # 경도
            else:
                print(address)
        # end if

        # 올바른 위도 경도 형식이 아니면
        if latLong[1] == '0' or latLong[0] == '0':            
            # 카카오 지도 api 이용하여 위도와 경도를 취득합니다.
            geoInfo = getGeoCoder(address) 
            if geoInfo != None:
                latitude = geoInfo[0] # 위도
                longitude = geoInfo[1] # 경도
            else:
                print(address)
        # end if
        
        ediyaData.append([brand, name, address, sido, gu, latitude, longitude, geoInfoImsi])        
        
        # break
    # print('-'*30)
    
# end for    

print('이디야 매장 개수 : %d' % len(ediyaData))

  0%|          | 0/25 [00:00<?, ?it/s]

서울 노원구 한글비석로 409 (상계동) 1~2층
서울 노원구 한글비석로 409 (상계동) 1~2층
서울 동작구 사당로16가길 96 (사당동) 1,2층
서울 동작구 사당로16가길 96 (사당동) 1,2층
서울 마포구 백범로 100 (대흥동)
서울 마포구 백범로 100 (대흥동)
서울 영등포구 영등포로35길 19 (영등포동6가)
서울 영등포구 영등포로35길 19 (영등포동6가)
이디야 매장 개수 : 573


In [32]:
ediyaDataFrame = pd.DataFrame(ediyaData)
ediyaDataFrame.columns = ['브랜드', '상호', '주소', '시도', '군구', '위도', '경도', 'geoInfoImsi']
ediyaDataFrame.head()

,브랜드,상호,주소,시도,군구,위도,경도,geoInfoImsi
0,이디야,금란망우점,서울 중랑구 망우로 460 (망우동),서울,중랑구,37.6001065609187,127.103136691889,"panLatTo('0','0','0');fnMove();"
1,이디야,동원사거리점,"서울 중랑구 겸재로 240 (면목동, 행복오피스텔)",서울,중랑구,37.5896269575279,127.094182772191,"panLatTo('127.094182772191','37.5896269575279'..."
2,이디야,망우동점,서울 중랑구 망우로 416 (망우동),서울,중랑구,37.5991657770242,127.098351137589,"panLatTo('0','0','2');fnMove();"
3,이디야,망우중앙점,"서울 중랑구 용마산로115길 109 (망우동, 한일써너스빌리젠시2단지)",서울,중랑구,37.5974674047065,127.09415879594557,"panLatTo('127.09415879594557','37.597467404706..."
4,이디야,망우코레일점,"서울 중랑구 망우로55길 11-10 (상봉동, 망우역)",서울,중랑구,37.5992876153903,127.092756577852,"panLatTo('127.092756577852','37.5992876153903'..."


In [23]:
print('위도 누락 데이터 개수 : %d' % ediyaDataFrame['위도'].isnull().sum())
print('경도 누락 데이터 개수 : %d' % ediyaDataFrame['경도'].isnull().sum())

위도 누락 데이터 개수 : 4
경도 누락 데이터 개수 : 4


In [24]:
print('스타 벅스 매장 개수 : %d' % len(sbDataFrame))
print('이디야 매장 개수 : %d' % len(ediyaDataFrame))

스타 벅스 매장 개수 : 615
이디야 매장 개수 : 549


In [33]:
# 임시 컬럼을 제거하고, 2 매장의 데이터 프레임을 합칩니다.
ediyaDataFrame = ediyaDataFrame.drop('geoInfoImsi', axis=1)
coffeeFrame = pd.concat([sbDataFrame, ediyaDataFrame])
print('전체 매장 개수 : %d' % len(coffeeFrame))

전체 매장 개수 : 1188


In [38]:
coffeeFrame['시도'].unique()

array(['서울특별시', '서울'], dtype=object)

In [39]:
# 보정을 위한 텍스트 파일을 읽어서, 사전을 생성합니다.
sido_txt = open('sido_dict.txt', mode='rt', encoding='UTF-8')
sido_dict = dict()

sido_list = sido_txt.readlines()
print(type(sido_list))

for oneline in sido_list:
    # print('[' + oneline + ']')
    mydata = oneline.replace('\n', '').split(':')
    sido_dict[mydata[0]] = mydata[1]

print(sido_dict)

<class 'list'>
{'서울': '서울특별시', '서울시': '서울특별시', '진해': '창원', '마산': '창원', '부산': '부산광역시', '부산시': '부산광역시'}


In [40]:
coffeeFrame.loc[:, '시도'] = coffeeFrame.loc[:, '시도'].apply(lambda xxx : sido_dict.get(xxx, xxx))
coffeeFrame['시도'].unique()

array(['서울특별시'], dtype=object)

In [43]:
# 위도에 누락된 매장 정보 확인
missing_latitude = coffeeFrame[coffeeFrame['위도'].isnull()]
missing_latitude

,브랜드,상호,주소,시도,군구,위도,경도
170,이디야,상계역사거리점,서울 노원구 한글비석로 409 (상계동) 1~2층,서울특별시,노원구,NaN,NaN
219,이디야,총신대점,"서울 동작구 사당로16가길 96 (사당동) 1,2층",서울특별시,동작구,NaN,NaN
223,이디야,대흥역점,서울 마포구 백범로 100 (대흥동),서울특별시,마포구,NaN,NaN
388,이디야,영등포시장점,서울 영등포구 영등포로35길 19 (영등포동6가),서울특별시,영등포구,NaN,NaN


In [44]:
# 위도 경도를 수동으로 넣어 주기 작업
coffeeFrame.loc[170, ['위도', '경도']] = [37.5, 127.05]
coffeeFrame.loc[219, ['위도', '경도']] = [37.5, 127.05]
coffeeFrame.loc[223, ['위도', '경도']] = [37.5, 127.05]
coffeeFrame.loc[388, ['위도', '경도']] = [37.5, 127.05]

In [46]:
filename = 'coffeeList.csv'
# coffeeFrame.to_csv(filename, encoding='CP949', index=False) # 엑셀에서 한글 안깨지고 보려면
coffeeFrame.to_csv(filename, encoding='UTF-8', index=True) # 일반적인 방법
print(filename + ' 파일 저장됨')

coffeeList.csv 파일 저장됨


In [75]:
coffeeFrame.columns

Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도'], dtype='object')

In [81]:
# 지도 그리기 : '군구' 컬럼이 '마포구', '용산구', '서대문구'인 항목들만 추출
mapFrame = coffeeFrame[coffeeFrame['군구'].isin(['마포구', '용산구', '서대문구'])]
print(len(mapFrame))

138


In [79]:
# 위도와 경도를 숫자 형식으로 형변환합니다.
mapFrame['위도'] = pd.to_numeric(mapFrame['위도'], errors='coerce')
mapFrame['경도'] = pd.to_numeric(mapFrame['경도'], errors='coerce')

C:\Users\ict-00\AppData\Local\Temp\ipykernel_1532\640425397.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapFrame['위도'] = pd.to_numeric(mapFrame['위도'], errors='coerce')
C:\Users\ict-00\AppData\Local\Temp\ipykernel_1532\640425397.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapFrame['경도'] = pd.to_numeric(mapFrame['경도'], errors='coerce')


In [91]:
latitude = mapFrame['위도']
print(type(latitude))
print('총합 : ' + str(latitude.sum()))
print('최소 : ' + str(latitude.min()))
print('최대 : ' + str(latitude.max()))
print('평균 : ' + str(latitude.mean()))
print('중앙값 : ' + str(latitude.median()))
print('표준 편차 : ' + str(latitude.std()))

<class 'pandas.core.series.Series'>
총합 : 5182.291040841956
최소 : 37.5
최대 : 37.594570187466
평균 : 37.55283362928954
중앙값 : 37.55423527802193
표준 편차 : 0.016978046271204115


In [97]:
map_center = [mapFrame['위도'].mean(), mapFrame['경도'].mean()]
print(map_center)
mapData = folium.Map(location=map_center, zoom_start=11.5, control_scale=True, title='openstreepmap')

[np.float64(37.55283362928954), np.float64(126.94369741275428)]


In [99]:
for idx, row in mapFrame.iterrows():
    # 브랜드별 마커 색상 설정
    if row['브랜드'] == '이디야':
        mk_color = 'darkblue' # 마커 색상
        ic_color = 'white' # 아이콘 색상
    elif row['브랜드'] == '스타벅스':
        mk_color = 'green'
        ic_color = 'white'
    # end if    

    # 지도에 마커 생성하기
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup = row['주소'],
        tooltip = row['브랜드'],
        icon = folium.Icon(
            color = mk_color,
            icon_color = ic_color,
            icon = 'coffee',
            prefix = 'fa'
        )
    ).add_to(mapData)    
# end for    

In [100]:
mapData